In [ ]:
import os
import shutil
from deepface import DeepFace

# --- CONFIGURAÇÕES DE CAMINHOS ---
PASTA_CONHECIDOS = r"CAMINHO PARA A PASTA\Base_dados\imagens_conhecidas"
PASTA_ORIGEM = r"CAMINHO PARA A PASTA\Base_dados\Imagens_Para_Analisar"
PASTA_SAIDA_BASE = r"CAMINHO PARA A PASTA\Base_dados\Rostos Detectados"
PASTA_NAO_ENCONTRADOS = r"CAMINHO PARA A PASTA\Base_dados\nao encontrados"

# --- CONFIGURAÇÕES DO MODELO ---
MODELO = "ArcFace" 
DETECTOR = "retinaface" 
THRESHOLD = 0.30

In [ ]:

def organizar_fotos():
    # 1. Pergunta inicial ao usuário
    pergunta = input("Gostaria de fazer uma separação otimizada? (Imagens encontradas não serão analisadas novamente) [S/N]: ").strip().upper()
    otimizado = True if pergunta == "S" else False

    # Cria as pastas base se não existirem
    os.makedirs(PASTA_SAIDA_BASE, exist_ok=True)
    os.makedirs(PASTA_NAO_ENCONTRADOS, exist_ok=True)

    # Lista as imagens conhecidas (referências), caso tenha outros formatos, pode ser necessário ajustar a extensão
    fotos_conhecidas = [f for f in os.listdir(PASTA_CONHECIDOS) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Lista as imagens para analisar
    Imagens_Para_Analisar = [f for f in os.listdir(PASTA_ORIGEM) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Conjunto para rastrear o que já foi encontrado (para a pasta "não encontrados" e otimização)
    encontrados_geral = set()

    print(f"\n--- Iniciando Processamento ---")
    print(f"Modo Otimizado: {'ATIVADO' if otimizado else 'DESATIVADO'}")
    print(f"Pessoas conhecidas: {len(fotos_conhecidas)}")
    print(f"Total de imagens para analisar: {len(Imagens_Para_Analisar)}\n")

    # Loop por cada pessoa conhecida
    for foto_ref in fotos_conhecidas:
        nome_pessoa = os.path.splitext(foto_ref)[0] # Pega o nome do arquivo sem extensão
        caminho_ref = os.path.join(PASTA_CONHECIDOS, foto_ref)
        
        # Cria uma pasta específica para esta pessoa
        pasta_destino_pessoa = os.path.join(PASTA_SAIDA_BASE, nome_pessoa)
        os.makedirs(pasta_destino_pessoa, exist_ok=True)
        
        print(f">> Procurando por: {nome_pessoa}...")

        # Lista de imagens que vamos iterar (se for otimizado, filtramos o que já foi achado)
        lista_atual = [img for img in Imagens_Para_Analisar if img not in encontrados_geral] if otimizado else Imagens_Para_Analisar

        for arquivo in lista_atual:
            caminho_alvo = os.path.join(PASTA_ORIGEM, arquivo)
            
            try:
                resultado = DeepFace.verify(
                    img1_path = caminho_ref,
                    img2_path = caminho_alvo,
                    model_name = MODELO,
                    detector_backend = DETECTOR,
                    enforce_detection = False,
                    align = True,
                    threshold = THRESHOLD#aqui é a distância máxima para considerar um match, quanto menor, mais rigoroso!!!
                )

                if resultado['verified']:
                    shutil.copy(caminho_alvo, os.path.join(pasta_destino_pessoa, arquivo))
                    encontrados_geral.add(arquivo)
                    print(f"   [+] Encontrado {nome_pessoa} em {arquivo} (Dist: {resultado['distance']:.4f})")

            except Exception as e:
                print(f"\nErro ao processar {arquivo} com {nome_pessoa}: {e}")

    # --- FINALIZAÇÃO: Mover não encontrados ---
    print("\n--- Finalizando: Organizando imagens não reconhecidas ---")# aqui provavelmente é onde ficaram as imagens em multidão, alterar depois
    nao_encontrados_count = 0
    for img in Imagens_Para_Analisar:
        if img not in encontrados_geral:
            shutil.copy(os.path.join(PASTA_ORIGEM, img), os.path.join(PASTA_NAO_ENCONTRADOS, img))
            nao_encontrados_count += 1

    print(f"\nProcessamento concluído!")
    print(f"Imagens que não deram match: {nao_encontrados_count}")

if __name__ == "__main__":
    organizar_fotos()


--- Iniciando Processamento ---
Modo Otimizado: ATIVADO
Pessoas conhecidas: 10
Total de imagens para analisar: 212

>> Procurando por: user01...
   [+] Encontrado user01 em _MG_1440.JPG (Dist: 0.0898)
   [+] Encontrado user01 em _MG_1441.JPG (Dist: 0.0000)
   [+] Encontrado user01 em _MG_1442.JPG (Dist: 0.1217)
   [+] Encontrado user01 em _MG_1443.JPG (Dist: 0.1880)


KeyboardInterrupt: 